In [24]:
import requests
from urllib.request import urlopen
from urllib.parse import urlencode,unquote,quote_plus
import urllib
import lxml
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

In [25]:
판례목록_df = pd.read_html("법령검색목록.html")
판례목록_df = 판례목록_df[0]
판례목록_df

,순번,판례ID,제목,법원,사건유형,판결유형,선고일자
0,1,216815,가축분뇨배출시설변경허가신청불허가처분취소청구,대법원,대법원,일반행정,2021.06.30
1,2,216821,강도상해,대법원,대법원,형사,2021.06.30
2,3,216819,강제추행(인정된 죄명: 공연음란),대법원,대법원,형사,2021.06.30
3,4,216807,대여금,대법원,대법원,민사,2021.06.30
4,5,216803,손해배상(기),대법원,대법원,민사,2021.06.30
...,...,...,...,...,...,...,...
39995,39996,103550,거절사정,대법원,대법원,특허,1987.09.08
39996,39997,103535,건물철거,대법원,대법원,민사,1987.09.08
39997,39998,103544,건물철거등,대법원,대법원,민사,1987.09.08
39998,39999,103581,"공문서위조,공문서위조행사,사문서위조행사,공정증서원본불실기재행사,공정증서원본불실기재,...",대법원,대법원,형사,1987.09.08


In [26]:
info_df = pd.read_excel("id_판시사항,판결요지,참조조문,참조판례_판례전체_주문(칼럼따로).xlsx")
info_df["판시사항"].replace("",np.nan,inplace=True)
info_df["판시사항"].replace(" ","",inplace=True)
info_df["판시사항"].replace("\n","",inplace=True)
info_df = info_df.drop(info_df[info_df["판시사항"].isna()].index)
# info_법령전체_df.to_excel("id_판시사항,판결요지,참조조문,참조판례_판례전체.xlsx",index=False)
info_df

,ID,판시사항,판결요지,참조조문,참조판례,주문
0,150093,[1] \n \n 구 임대주택법 시행규칙 제3조의3...,[1] 법령의 규정이 특정 행정기관에게 그 법령 내용의 구체적 사항을 정할 수 있는...,[1] \n \n 구 임대주택법 시행규칙(2008. 6. ...,[1] \n \n 대법원 2003. 9. 26. 선고 20...,"1. 당심에서 교환적으로 변경된 청구에 따라, 피고는 각 원고에게 별지 2 분양..."
1,209310,연안복합어선의 소유자 겸 선장인 甲이 잠수부 乙을 승선시킨 후 출항하였다가 선착장 ...,연안복합어선의 소유자 겸 선장인 甲이 잠수부 乙을 승선시킨 후 출항하였다가 선착장 ...,"상법 제879조 제2항, 자동차손해배상 보장법 제3조",NaN,"1. 제1심판결을 다음과 같이 변경한다. 가. 피고는, 1) 원고 1에게 ..."
2,208911,甲 지방자치단체가 관리기관인 농공단지의 안에 위치한 공장에서 콘크리트제품 제조업을 ...,甲 지방자치단체가 관리기관인 농공단지의 안에 위치한 공장에서 콘크리트제품 제조업을 ...,"국가배상법 제2조, 행정절차법 제2조 제3호, 제48조, 산업집적활성화 및 공장설립...",NaN,"1. 제1심판결 중 피고 패소 부분을 취소하고, 그 취소 부분에 해당하는 원고..."
3,208459,국내외 항공운송업 등을 목적으로 하는 甲 주식회사가 비행장을 설치하여 관리·운영하고...,국내외 항공운송업 등을 목적으로 하는 甲 주식회사가 비행장을 설치하여 관리·운영하고...,"민법 제214조, 민사집행법 제300조",NaN,1. 이 사건 항고를 기각한다. 2. 항고비용은 채권자가 부담한다.
4,208460,아이돌봄 지원법에 따라 甲 지방자치단체의 각 구청장이 관할 구역에 지정한 서비스제공...,아이돌봄 지원법에 따라 甲 지방자치단체의 각 구청장이 관할 구역에 지정한 서비스제공...,"근로기준법 제2조 제1항 제1호, 아이돌봄 지원법 제2조 제3호, 제4호, 제11조...",NaN,"항소인 1. 제1심판결 중 원고 40, 원고 53, 원고 110을 제외한 나머지..."
...,...,...,...,...,...,...
24680,103548,"대법원의 결정에 대한 항고, 특별항고의 가부",대법원의 결정에 대하여는 항고나 특별항고를 할 수 없다.,민사소송법 제420조,"대법원 1970.1.17 자 69그22 결정, \n 1982.10.22...",이 사건 항고를 각하한다.
24681,211021,"대법원의 결정에 대한 항고, 특별항고의 가부",대법원의 결정에 대하여는 항고나 특별항고를 할 수 없다.,민사소송법 제420조,"대법원 1970.1.17 자 69그22 결정, 1982.10.22 자 82그23 결정",이 사건 항고를 각하한다.
24682,103542,가. \n 국유재산법 제51조 제1항의 규정취지와 적용요건\n ...,가. \n 국유재산법 제51조 제1항은 국유재산에 대한 점유나 사용...,"국유재산법 제51조 제1항, \n 민사소송법 제197조 제1항",나. \n 대법원 1979.2.13. 선고 78다2107 판결,상고를 모두 기각한다. 상고비용은 각자 부담으로 한다.
24683,103535,가. 부동산 매매계약에 있어서 매도인이 매수인에게 지체책임을 지워 매매계약을 해제하...,가. 쌍무계약은 부동산매매계약에 있어서는 특별한 사정이 없는 한 매수인의 잔대금 지...,민법 제544조,"가. \n 대법원 1969.9.23. 선고 69다804,805 판결, ...","원심판결을 파기하고, 사건을 서울민사지방법원 합의부에 환송한다."


In [27]:
posts = info_df.get("판시사항")
posts

0        [1] \n          \n          구 임대주택법 시행규칙 제3조의3...
1        연안복합어선의 소유자 겸 선장인 甲이 잠수부 乙을 승선시킨 후 출항하였다가 선착장 ...
2        甲 지방자치단체가 관리기관인 농공단지의 안에 위치한 공장에서 콘크리트제품 제조업을 ...
3        국내외 항공운송업 등을 목적으로 하는 甲 주식회사가 비행장을 설치하여 관리·운영하고...
4        아이돌봄 지원법에 따라 甲 지방자치단체의 각 구청장이 관할 구역에 지정한 서비스제공...
                               ...                        
24680                             대법원의 결정에 대한 항고, 특별항고의 가부
24681                             대법원의 결정에 대한 항고, 특별항고의 가부
24682    가. \n          국유재산법 제51조 제1항의 규정취지와 적용요건\n   ...
24683    가. 부동산 매매계약에 있어서 매도인이 매수인에게 지체책임을 지워 매매계약을 해제하...
24684                                           권리남용의 성립요건
Name: 판시사항, Length: 24677, dtype: object

In [28]:
# 불용어 리스트
# 불용어, 불필요 단어 제거
stop_words_df = pd.read_excel("stopwords.xlsx")
stop_words_df

,불용어
0,아
1,휴
2,아이구
3,아이쿠
4,아이고
...,...
702,점
703,증
704,바
705,운


In [29]:
from konlpy.tag import Okt
okt = Okt()

oo = okt.pos(posts[0],
        norm=True,   # 정규화(normalization)
        stem=True    # 어간추출(stemming)
        )
print(oo)

[('[', 'Punctuation'), ('1', 'Number'), (']', 'Punctuation'), ('구', 'Noun'), ('임대주택', 'Noun'), ('법', 'Noun'), ('시행', 'Noun'), ('규칙', 'Noun'), ('제', 'Noun'), ('3조', 'Number'), ('의', 'Foreign'), ('3', 'Number'), ('제', 'Noun'), ('1', 'Number'), ('항', 'Noun'), ('에', 'Josa'), ('정', 'Noun'), ('한', 'Josa'), ('‘', 'Foreign'), ('공공', 'Modifier'), ('건설', 'Noun'), ('임대주택', 'Noun'), ('분양', 'Noun'), ('전환', 'Noun'), ('가격', 'Noun'), ('의', 'Josa'), ('산정', 'Noun'), ('기준', 'Noun'), ('’', 'Punctuation'), ('의', 'Noun'), ('법적', 'Noun'), ('성질', 'Noun'), ('(=', 'Punctuation'), ('법규명령', 'Noun'), (')', 'Punctuation'), ('\n        \n        ', 'Foreign'), ('[', 'Punctuation'), ('2', 'Number'), (']', 'Punctuation'), ('대', 'Modifier'), ('한', 'Modifier'), ('주택', 'Noun'), ('공사', 'Noun'), ('가', 'Josa'), ('‘', 'Foreign'), ('공공', 'Modifier'), ('건설', 'Noun'), ('임대주택', 'Noun'), ('분양', 'Noun'), ('전환', 'Noun'), ('가격', 'Noun'), ('의', 'Josa'), ('산정', 'Noun'), ('기준', 'Noun'), ('’', 'Punctuation'), ('에', 'Josa'), ('위', 'Noun'

In [30]:
from konlpy.tag import Mecab
# from eunjeon import Mecab
# 형태소 분석기 
tagger = Mecab()
stop_words = list(pd.read_excel("stopwords.xlsx").get("불용어"))
stop_words

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를 들면',
 '예를 들자면',
 '저',
 '소인',
 '소생',
 '저희',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수 없다',
 '해서는 안된다',
 '뿐만 아니라',
 '만이 아니다',
 '만은 아니다',
 '막론하고',
 '관계없이',
 '그치지 않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지 않다',
 '따지지 않다',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만 못하다',
 '하는 편이 낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이 외에',
 '이 밖에',
 '하여야',
 '비로소',
 '한다면 몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할 생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게 함으로써',
 '하지만',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에 틀림없다',
 '한다면',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
 '훨씬',
 '얼마나',
 '얼마만큼',
 '얼마큼',
 '남짓',
 '여',
 '얼마간',
 '약간',
 '다소',
 '좀',
 '조

In [31]:
def tokenizer_1(raw_texts, pos=["Noun","Alpha","Verb","Number"], stop_words=list(stop_words_df.get("불용어"))):
    p = okt.pos(raw_texts, 
            norm=True,   # 정규화(normalization)
            stem=True    # 어간추출(stemming)
            )
    o = [word for word, tag in p if len(word) > 1 and tag in pos and word[0] not in stop_words]
    return(o)

tokenizer_1(posts[80])


['쌍무계약', '채무자', '미리', '채무', '표시', '경우', '해당', '보다']

In [32]:
def tokenizer_2(raw_texts, pos=["Noun","Alpha","Verb","Number"], stop_words=list(stop_words_df.get("불용어"))):
    nouns = []
  
    for noun in tagger.nouns(raw_texts):
        if noun not in stop_words and len(noun)>1:
            nouns.append(noun)
    return nouns

tokenizer_2(posts[80])

['쌍무', '계약', '채무자', '채무', '이행', '의사', '표시', '사례']

# CountVectorizer(Okt)

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
vectorize = CountVectorizer(
    tokenizer=tokenizer_1, # 문장에 대한 tokenizer (위에 정의한 함수 이용)
    min_df=5,            # 단어가 출현하는 최소 문서의 개수
    stop_words= list(stop_words_df.get("불용어"))
)

X = vectorize.fit_transform(posts)
# X.toarray()
pd.DataFrame(X.toarray())

/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['갖다', '거리', '견지', '결과', '결론', '고려', '공동', '관계', '관련', '근거', '기대', '기점', '기준', '까닭', '낫다', '누가', '다시', '달다', '대로', '대하', '대해', '더불다', '더욱이', '도달', '도착', '동시', '돼다', '되다', '되어다', '두번째', '뒤따르다', '따다', '따르다', '따름', '라면', '리하', '마자', '막론', '만이', '만큼', '만하', '말다', '망정', '몰랏다', '무릎', '반대', '방면', '보다', '보아', '본대', '부류', '불구', '불문', '비걱거리', '비교', '비기다', '비추다', '상대', '생각', '서술', '시오', '시작', '시초', '쓰다', '쓰이다', '안다', '않다', '알다', '연관', '울다', '입다', '입장', '잇따르다', '정도', '종합', '주저', '즈음', '지경', '지다', '진짜', '차다', '켜다', '크다', '틈타다', '편이', '한적', '해도', '해보다', '형식'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


,0,1,2,3,4,5,6,7,8,9,...,4604,4605,4606,4607,4608,4609,4610,4611,4612,4613
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24672,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24673,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24674,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24675,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
count_df = pd.DataFrame(X.toarray())

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
# 2번째와 3번째 문서(유클리디안 거리 값)
print(euclidean_distances(X[1], X[2]))#, X)
# 2번째와 3번째 문서(코사인유사도 값)
print(cosine_similarity(X[1], X[2]))#, X)
# [[0.1837195]]

[[18.22086716]]
[[0.0957209]]


In [36]:
# target_id 와 유사한 판례를 cosine_similarity으로 계산하여 유사벡터 도출
# 약 25초 소요
cosine_result = {}
target_id = 150093
x = info_df[info_df['ID']==target_id].index[0]  # target_id의 Id값의 index
for y in range(len(info_df)):
    cosine_result[info_df['ID'].iloc[y]] = cosine_similarity(X[x], X[y])[0][0]
sorted_cosine_dic = sorted(cosine_result.items(), key = lambda x:x[1], reverse = True)

# cosine_similariry 값 기준으로 내림차순 정렬된 유사 판례들
# 먼저나올수록 유사한 판례이다.
sorted_cosine_dic 

[(150093, 0.9999999999999998),
 (149741, 0.8202690927719182),
 (161231, 0.8150910773486999),
 (167508, 0.7312724241271306),
 (143913, 0.7253185207353656),
 (214417, 0.6929595462854152),
 (157933, 0.6562179588897105),
 (179722, 0.6512554904335259),
 (161700, 0.6310934206284035),
 (198830, 0.618817104846334),
 (204890, 0.618817104846334),
 (116053, 0.6098367211363062),
 (196672, 0.6098367211363062),
 (119299, 0.5802187275158867),
 (197400, 0.5720775535473553),
 (150030, 0.5465154148595432),
 (118012, 0.5343667144834019),
 (190936, 0.5229554962835848),
 (79947, 0.5229554962835848),
 (177480, 0.5208568575229744),
 (70700, 0.5149286505444373),
 (200720, 0.5106885876246954),
 (178113, 0.5048252022715237),
 (131274, 0.5015945091376615),
 (68186, 0.49906038194003577),
 (140294, 0.4954336943068622),
 (142804, 0.4952274784001943),
 (212273, 0.49379046027992046),
 (71012, 0.4913892699623653),
 (214525, 0.4904253164307734),
 (70562, 0.48221974779913784),
 (175516, 0.4793084795011092),
 (170634, 0.

In [38]:
# target_id 에 해당하는 판례와 유사한 판례 상위 10개 도출
for id_ in sorted_cosine_dic[0:11]:
    url = "http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID={}&type=HTML".format(id_[0])
    url2 = "https://www.law.go.kr/precInfoP.do?mode=0&precSeq={}&vSct=*".format(id_[0])
    print(url)

http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=150093&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=149741&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=161231&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=167508&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=143913&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=214417&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=157933&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=179722&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=161700&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=198830&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=204890&type=HTML


# CountVectorizer (Mecab)

In [39]:
from sklearn.feature_extraction.text import CountVectorizer
vectorize = CountVectorizer(
    tokenizer=tokenizer_2, # 문장에 대한 tokenizer (위에 정의한 함수 이용)
    min_df=5,            # 단어가 출현하는 최소 문서의 개수
    stop_words= list(stop_words_df.get("불용어"))
)

X = vectorize.fit_transform(posts)
# X.toarray()
pd.DataFrame(X.toarray())

/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['개의', '거니', '견지', '결과', '결론', '고려', '고용', '공동', '관계', '관련', '구체', '구토', '근거', '기관', '기점', '기준', '까닭', '나다', '나머지', '노동', '노동부', '누가', '대로', '도달', '도착', '동시', '따름', '때문', '마음', '방면', '부령', '부류', '불문', '비교', '사람', '상대', '생각', '서술', '시작', '시초', '아래', '아무', '연관', '의거', '의지', '이외', '이용', '이유', '일반', '일지', '입각', '입장', '정도', '제외', '종합', '주저', '즈음', '지경', '진짜', '필요', '한적', '할만', '항목', '행정', '형식'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


,0,1,2,3,4,5,6,7,8,9,...,5158,5159,5160,5161,5162,5163,5164,5165,5166,5167
0,0,0,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24672,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24673,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24674,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24675,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
count_df = pd.DataFrame(X.toarray())

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
# 2번째와 3번째 문서(유클리디안 거리 값)
print(euclidean_distances(X[1], X[2]))#, X)
# 2번째와 3번째 문서(코사인유사도 값)
print(cosine_similarity(X[1], X[2]))#, X)
# [[0.1837195]]

[[22.56102835]]
[[0.05960861]]


In [41]:
# target_id 와 유사한 판례를 cosine_similarity으로 계산하여 유사벡터 도출
# 약 25초 소요
cosine_result = {}
target_id = 150093
x = info_df[info_df['ID']==target_id].index[0]  # target_id의 Id값의 index
for y in range(len(info_df)):
    cosine_result[info_df['ID'].iloc[y]] = cosine_similarity(X[x], X[y])[0][0]
sorted_cosine_dic = sorted(cosine_result.items(), key = lambda x:x[1], reverse = True)

# cosine_similariry 값 기준으로 내림차순 정렬된 유사 판례들
# 먼저나올수록 유사한 판례이다.
sorted_cosine_dic 

[(150093, 0.9999999999999998),
 (161231, 0.8661113147418232),
 (149741, 0.8613762496087309),
 (167508, 0.7888764815965523),
 (214417, 0.786183526401858),
 (143913, 0.7734747151919387),
 (179722, 0.7008766440504626),
 (157933, 0.7000624711815735),
 (161700, 0.6705129799575595),
 (119299, 0.6680767400622813),
 (116053, 0.6629935441317959),
 (196672, 0.6629935441317959),
 (197400, 0.6355171025060142),
 (198830, 0.6236413804652249),
 (204890, 0.6236413804652249),
 (172356, 0.6122870221469174),
 (178019, 0.567653032368184),
 (179704, 0.5580516733397745),
 (177480, 0.5562342005550127),
 (175516, 0.5560597575690315),
 (166966, 0.5493189389270062),
 (184547, 0.532346557369217),
 (70562, 0.5210343884781828),
 (198973, 0.49888765156985887),
 (200720, 0.4983952887201328),
 (165919, 0.49650955593063345),
 (133922, 0.48795003647426666),
 (199888, 0.48688425427022897),
 (124977, 0.483045891539648),
 (166801, 0.48141121378376395),
 (169234, 0.47929833168529024),
 (178113, 0.4783403848824849),
 (71443

In [42]:
# target_id 에 해당하는 판례와 유사한 판례 상위 10개 도출
for id_ in sorted_cosine_dic[0:11]:
    url = "http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID={}&type=HTML".format(id_[0])
    url2 = "https://www.law.go.kr/precInfoP.do?mode=0&precSeq={}&vSct=*".format(id_[0])
    print(url)

http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=150093&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=161231&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=149741&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=167508&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=214417&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=143913&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=179722&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=157933&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=161700&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=119299&type=HTML
http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=116053&type=HTML
